# MATCH

This notebook provides wrapper functions for calling the [MATCH algorithm](https://www.eecs.qmul.ac.uk/~simond/match/).  Running this algorithm requires installing some other software, which is described below.  This notebook implements the `offline_processing()` and `online_processing()` functions, which will be imported and run in `02_RunExperiment.ipynb`.

Here is a summary of the source separated MATCH approach:
- Offline processing:
    - The source separation is used to split the full mix recording into solo piano and orchestra recordings.
    - The orchestra and estimated orchestra are aligned with standard DTW using chroma features.
- Online processing: The solo piano and estimated piano are aligned with the MATCH algorithm, and the predicted alignment is then used to infer the corresponding alignment between the piano and orchestra recordings.

## Offline Processing

The offline processing is the same as in the simple offline DTW system.  In the offline processing stage, three things are computed and stored in the `cache/` folder:
- chroma features for the orchestra recording
- chroma features for the full mix recording
- predicted DTW alignment between the orchestra and full mix recordings

NOTE: because we do not have the code to do the source separation, the pre-source separated files need to be added to the cache folder BEFORE this code can be run. This is an example of how the cache folder should look *before* running the offline processing (assuming you wanted to use the HDemucs source separation model):


```
cache
│   └── separation
│       ├── HDemucs
│       │   ├── bach5_mov1_PO1_O.wav
│       │   ├── bach5_mov1_PO1_P.wav
│       │   ├── bach5_mov1_PO2_O.wav
│       │   ├── bach5_mov1_PO2_P.wav
│       │   ├── beeth1_mov1_PO1_O.wav
│       │   ├── beeth1_mov1_PO1_P.wav
│       │   ├── beeth1_mov1_PO2_O.wav
│       │   ├── beeth1_mov1_PO2_P.wav
│       │   ├── mozart21_mov1_PO1_O.wav
│       │   ├── mozart21_mov1_PO1_P.wav
│       │   ├── mozart21_mov1_PO2_O.wav
│       │   ├── mozart21_mov1_PO2_P.wav
│       │   ├── rach2_mov1_PO1_O.wav
│       │   ├── rach2_mov1_PO1_P.wav
│       │   ├── rach2_mov1_PO2_O.wav
│       │   └── rach2_mov1_PO2_P.wav
```

In [1]:
import numpy as np
import pandas as pd
import import_ipynb
import librosa as lb
import system_utils
import align_tools
import System_MATCH
import os
import os.path
import subprocess
from pathlib import Path

importing Jupyter notebook from system_utils.ipynb
importing Jupyter notebook from align_tools.ipynb
importing Jupyter notebook from System_MATCH.ipynb
importing Jupyter notebook from System_OfflineDTW.ipynb
importing Jupyter notebook from sonify_tools.ipynb
importing Jupyter notebook from tsm_tools.ipynb


In [2]:
def verify_separated_file(indir, piece):
    '''Verifies that the specified directory has the separated files.
    
    Args:
        indir (pathlib.Path): The directory to verify
        piece (str): The file to check for
    '''
    separated_filename_p = piece + '_P.wav'
    separated_filename_o = piece + '_O.wav'
    
    assert (indir / separated_filename_p).exists(), f'Missing separated file {separated_filename_p}'
    assert (indir / separated_filename_o).exists(), f'Missing separated file {separated_filename_o}'

In [3]:
def offline_processing(scenario_dir, cache_dir, hop_length, steps, weights, separation_dir, separation_alg):
    '''Carries out the same offline processing steps as the simple offline DTW system.
    
    Args
        scenario_dir: The scenario directory to process
        cache_dir: The location of the cache directory
        hop_length: The hop length in samples used when computing chroma features
        steps: an L x 2 array specifying the allowable DTW transitions
        weights: a length L array specifying the DTW transition weights
        separation_dir: directory where the pre-separated files are stored
        separation_alg: algorithm to separate the files with (must be a subdirectory within `separation_dir`)
    
    This function will store the computed chroma features and estimated alignment in the cache folder.
    '''

    # setup
    system_utils.verify_scenario_dir(scenario_dir)

    if os.path.exists(cache_dir):
        # print(f'{cache_dir} has already been processed.  Skipping.')
        pass
    else:
        # TODO: move this below `os.makedirs` and do the separation here instead of importing pre-separated files
        separated_file_dir = Path(separation_dir) / 'separation' / Path(separation_alg)
        piece_name = Path(cache_dir).name.split('_')
        piece_name.pop(2)
        piece_name = '_'.join(piece_name)
        verify_separated_file(separated_file_dir, piece_name)

        # setup
        os.makedirs(cache_dir)

        # compute orchestra features
        o_file = f'{scenario_dir}/o.wav'
        y_o, sr = lb.core.load(o_file)
        F_o = lb.feature.chroma_cqt(y=y_o, sr=sr, hop_length=hop_length, norm=2) 

        # compute full mix separated orchestra features
        po_o_file = separated_file_dir / (piece_name+'_O.wav')
        y_po_o, sr = lb.core.load(po_o_file)
        F_po_o = lb.feature.chroma_cqt(y=y_po_o, sr=sr, hop_length=hop_length, norm=2)
      
        # compute subsequence DTW alignment (orchestra as query) 
        orch_start_sec, orch_end_sec = system_utils.get_orchestra_start_end_times(scenario_dir)
        orch_start_frm = int(np.round(orch_start_sec * sr / hop_length))
        orch_end_frm = int(np.round(orch_end_sec * sr / hop_length)) + 1
        wp = align_tools.compute_dtw_alignment(1 - F_o[:,orch_start_frm:orch_end_frm].T @ F_po_o, steps, weights, subseq = True)
        wp[0,:] = wp[0,:] + orch_start_frm  # account for offset

        # save to cache
        np.save(f'{cache_dir}/o_chroma.npy', F_o)
        np.save(f'{cache_dir}/po_o_chroma.npy', F_po_o)
        np.save(f'{cache_dir}/o_po_align.npy', wp)
    return

In [4]:
def verify_cache_dir(indir):
    '''
    Verifies that the specified cache directory has the required files.
    
    Inputs
    indir: The cache directory to verify
    '''

    # Feature Files
    assert os.path.exists(f'{indir}/o_chroma.npy'), f'Missing o_chroma.npy in {indir}'
    assert os.path.exists(f'{indir}/po_o_chroma.npy'), f'Missing po_o_chroma.npy in {indir}'
    assert os.path.exists(f'{indir}/o_po_align.npy'), f'Missing o_po_align.npy in {indir}'

## Online Processing

In the online processing stage, we do two things:
1. compute an online alignment between the piano and estimated piano using MATCH,
2. use the predicted alignment to infer the alignment between the piano and orchestra recordings

Note that step 1 is completed before we begin step 2.  This implementation is thus not a valid online system, but its performance nonetheless can tell us how well an online system would perform.

### Software Installation

Using the MATCH algorithm requires a few pieces of software to be installed:
- [Sonic Annotator](https://vamp-plugins.org/sonic-annotator/), a program for command-line processing of audio files
- [the MATCH Vamp plugin](https://code.soundsoftware.ac.uk/projects/match-vamp/), an implementation of the MATCH algorithm which can be used in tandem with Sonic Annotator
- the SoX command line audio utility tool

Below, we will assume that the `sonic-annotator` and `sox` binaries can be called from command line, and that the MATCH Vamp plugin has been installed.  See [here](https://vamp-plugins.org/download.html#install) for instructions on how to install Vamp plugins.

### Wrapper Implementation

In [5]:
def online_processing(scenario_dir, out_dir, cache_dir, hop_sec, separation_dir, separation_alg, oracle = False):
    '''
    Carries out `online' processing using the MATCH algorithm.
    
    Inputs
    scenario_dir: The scenario directory to process
    out_dir: The directory to put results, intermediate files, and logging info
    cache_dir: The cache directory
    hop_sec: The hop size in sec used in the offline DTW stage
    separation_dir: directory where the pre-separated files are stored
    separation_alg: algorithm to separate the files with (must be a subdirectory within `separation_dir`)
    oracle: boolean specifying if oracle information for query ending time should be used

    This function will compute and save the predicted alignment in the output directory in a file hyp.npy
    '''
    # TODO: move this below `os.makedirs` and do the separation here instead of importing pre-separated files
    separated_file_dir = Path(separation_dir) / 'separation' / Path(separation_alg)
    piece_name = Path(cache_dir).name.split('_')
    piece_name.pop(2)
    piece_name = '_'.join(piece_name)
    verify_separated_file(separated_file_dir, piece_name)
    
    # verify & setup
    System_MATCH.verify_match_installation()
    system_utils.verify_scenario_dir(scenario_dir)
    verify_cache_dir(cache_dir)
    assert not os.path.exists(out_dir), f'Output directory {out_dir} already exists.'
    os.makedirs(out_dir)
           
    # determine the start time of the query in the orchestra recording (ground truth)
    orch_start_sec, orch_end_sec = system_utils.get_orchestra_query_boundaries(scenario_dir)
    
    # infer the start time of the query in the full mix recording (estimated)
    wp_BC_frm = np.flipud(np.load(f'{cache_dir}/o_po_align.npy'))
    wp_BC_frm = np.hstack((np.array([0,0]).reshape((2,-1)), wp_BC_frm)) # prepend (0,0) to handle edge cases properly
    wp_BC_sec = wp_BC_frm * hop_sec
    fullmix_start_sec = np.interp(orch_start_sec, wp_BC_sec[1,:], wp_BC_sec[0,:])    
    
    # create audio recording of full mix containing the region of interest
    fullmix_orig_filepath = separated_file_dir / (piece_name+'_P.wav')
    fullmix_mod_filepath = f'{out_dir}/po_p_mod.wav'
    if oracle:
        # use both start and end locations
        fullmix_end_sec = np.interp(orch_end_sec, wp_BC_sec[1,:], wp_BC_sec[0,:]) # estimate end time of query in full mix
        # os.system(f'sox {fullmix_orig_filepath} {fullmix_mod_filepath} rate 22050 channels 1 trim {fullmix_start_sec} {fullmix_end_sec}')
        subprocess.run(['sox', fullmix_orig_filepath, fullmix_mod_filepath, 'rate', '22050', 'channels', '1', 'trim', str(fullmix_start_sec), str(fullmix_end_sec)],
                   check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    else:
        # only use start location (continue until end of recording)
        # os.system(f'sox {fullmix_orig_filepath} {fullmix_mod_filepath} rate 22050 channels 1 trim {fullmix_start_sec}')
        subprocess.run(['sox', fullmix_orig_filepath, fullmix_mod_filepath, 'rate', '22050', 'channels', '1', 'trim', str(fullmix_start_sec)],
                   check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    
    # run MATCH plugin
    piano_filepath = f'{scenario_dir}/p.wav'
    match_align_filepath = f'{out_dir}/match_p_po.out'
    # os.system(f'sonic-annotator -d vamp:match-vamp-plugin:match:b_a -m {piano_filepath} {fullmix_mod_filepath} -w csv --csv-stdout > {match_align_filepath}')    
    with open(match_align_filepath, 'w') as f:
        subprocess.run(['sonic-annotator', '-d', 'vamp:match-vamp-plugin:match:b_a', '-m', piano_filepath, fullmix_mod_filepath, '-w', 'csv', '--csv-stdout'],
                       check=True, stdout=f, stderr=subprocess.DEVNULL)
    
    # infer piano-orchestra alignment
    wp_AB_sec = System_MATCH.parse_match_outfile(match_align_filepath) # piano-fullmix alignment

    duration = lb.get_duration(path=piano_filepath)
    transposed = wp_AB_sec.transpose()
    transposed = transposed[transposed[:, 0] < duration].transpose()
    transposed[1,:] = transposed[1,:] + fullmix_start_sec # account for offset

    wp_AC_sec = align_tools.infer_alignment(transposed, wp_BC_sec) 
    np.save(f'{out_dir}/hyp.npy', wp_AC_sec)

    # save debugging info
    np.save(f'{out_dir}/p_po_align.npy', transposed)
    
    return

In [6]:
def verify_hyp_dir(indir):
    '''
    Verifies that the specified scenario hypothesis directory has the required files.
    
    Inputs
    indir: The cache directory to verify
    '''
    assert os.path.exists(f'{indir}/hyp.npy'), f'{indir} is missing the required files, please re run the online processing'

# Example

Here is an example of how to call the offline and online processing functions on a scenario directory.

In [ ]:
# scenario_dir = 'scenarios/s2'
# out_dir = 'experiments/test/s2'
# cache_dir = 'experiments/test/cache'
# hop_size = 512
# steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
# weights = np.array([2,3,3], dtype=np.float64)
# #offline_processing(scenario_dir, cache_dir, hop_size, steps, weights)
# online_processing(scenario_dir, out_dir, cache_dir, hop_size / 22050)